# Multithreading: writing multi-threaded programs
_COSC 208, Introduction to Computer Systems, 2023-04-28_

## Announcements
* Project 5 due Thursday, May 4

## Outline
* Warm-up
* Creating multiple threads
* Returning values from threads
* Passing multiple parameters to threads
* Practice writing multi-threaded programs

## Warm-up

Q1: _Consider the following program:_

In [ ]:
/* 1*/    #include <pthread.h>
/* 2*/    #include <stdio.h>
/* 3*/    #include <string.h>
/* 4*/    void word_count(char *str) {
/* 5*/        int count = 1;
/* 6*/        for (int i = 0; i < strlen(str); i++) {
/* 7*/            if (str[i] == ' ') {
/* 8*/                count++;
/* 9*/            }
/*10*/       }
/*11*/       printf("%d words\n", count);
/*12*/    }
/*13*/    int main(int argc, char *argv[]) {
/*14*/        char *str = "I love CS";
/*15*/        pthread_t thr;
/*16*/        pthread_create(thr, NULL, &word_count, str);
/*17*/        pthread_join(thr);
/*18*/    }

_Compiling this program with `gcc` results in the following errors:_

```
buggy_noreturn.c:16:20: warning: passing argument 1 of ‘pthread_create’ 
makes pointer from integer without a cast [-Wint-conversion]
   16 |     pthread_create(thr, NULL, &word_count, str);
      |                    ^~~
      |                    pthread_t {aka long unsigned int}
/usr/include/pthread.h:202:50: note: expected ‘pthread_t * restrict’ 
{aka ‘long unsigned int * restrict’} but argument is of type ‘pthread_t’ 
{aka ‘long unsigned int’}
  202 | extern int pthread_create (pthread_t *__restrict __newthread,
      |                            ~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~
buggy_noreturn.c:16:31: warning: passing argument 3 of ‘pthread_create’ 
from incompatible pointer type [-Wincompatible-pointer-types]
   16 |     pthread_create(thr, NULL, &word_count, str);
      |                               ^~~~~~~~~~~
      |                               void (*)(char *)
/usr/include/pthread.h:204:36: note: expected ‘void * (*)(void *)’ but 
argument is of type ‘void (*)(char *)’
  204 |                            void *(*__start_routine) (void *),
      |                            ~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~
buggy_noreturn.c:17:5: error: too few arguments to function ‘pthread_join’
   17 |     pthread_join(thr);
      |     ^~~~~~~~~~~~
/usr/include/pthread.h:219:12: note: declared here
  219 | extern int pthread_join (pthread_t __th, void **__thread_return);
      |            ^~~~~~~~~~~~
```

_How would you change the code to fix these problems?_

* Need to pass `&thr` to `pthread_create` (instead of `thr`) on line 16
* Function executed by a thread must return `void *` and take a single `void *` parameter; replace lines 6-7 with:
    ```C
    void *word_count(void *arg) {
        char *str = (char *)arg;
    ```
    Also add after line 11:
    ```C
    return NULL;
    ```
* Add an additional parameter to `pthread_join` one line 17:
    ```C
    pthread_join(thr, NULL);
    ``` 

## Creating multiple threads

* Create an array of `pthread_t` and an array of arguments
* Call `pthread_create` within a loop
* Call `pthread_join` within a separate loop
* Example

In [16]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#define NUM_THREADS 5
void *simple(void *arg) {
    int *id = (int *)arg;
    printf("I am thread %d\n", *id);
    return NULL;
}
int main() {
    pthread_t threads[NUM_THREADS];
    int ids[NUM_THREADS];
    for (int i = 0; i < NUM_THREADS; i++) {
        ids[i] = i+1;
        pthread_create(&(threads[i]), NULL, &simple, &(ids[i]));
    } 
    for (int i = 0; i < NUM_THREADS; i++) {
        pthread_join(threads[i], NULL);
    }
    printf("All threads finished\n");
}

I am thread 2
I am thread 3
I am thread 4
I am thread 5
I am thread 1
All threads finished


## Returning values from threads

* _When does a thread end?_ — when the function passed to `pthread_create` finishes (i.e., returns)
* _What happens to a function's parameters and local variables when the function returns?_ — they no longer exist (i.e., the stack frame is destroyed)
* _Where should we store a value that should exist even after a function returns?_ — on the heap
* Need to store a thread's return value on the heap
* Thread returns a pointer to the value on the heap
* Example

In [17]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <pthread.h>
void *length(void *arg) {
    char *str = (char *)arg;
    int *len = malloc(sizeof(int));
    *len = strlen(str);
    return len;
}
int main() {
    pthread_t thread;
    char *phrase = "Hello, threads!";
    pthread_create(&thread, NULL, &length, phrase);
    int *result = NULL;
    pthread_join(thread, (void **)&result);
    printf("Length: %d\n", *result);
    free(result);
}

Length: 15


* `pthread_join` returns `0` if successful, or an error number
* To get the pointer returned by the thread, we need to pass a location where the pointer can be stored — i.e., we need to pass a double pointer

## Passing multiple parameters to threads

* A thread can only take a single `void *` pointer as a parameter
* To pass multiple parameters:
    * Define a `struct` that contains a field for each of the parameters
    * Allocate space for an instance of the `struct`
    * Pass a pointer to the instance to the thread — i.e., as the last parameter in `pthread_create`

<div style="page-break-after:always;"></div>

## Practice writing multi-threaded programs

* Q2: _Write a function called `sum_array` which takes an array of `ARRAY_LEN` integers and returns the sum of the integers. Your function should have the appropriate prototype/implementation to serve as the entry point for a thread. Assume `ARRAY_LEN` is a constant which has been `#define`d._

<p style="height:20em;"></p>

* Q3: _Write a function called `sum_matrix` which takes an array of `NUM_ARRAYS` arrays of integers (i.e., an `int **`) and returns the sum of all the integers. The function should create `NUM_ARRAYS` threads, each running the `sum_array` function for a single array of integers. Assume `NUM_ARRAYS` is a constant which has been `#define`d._

<div style="page-break-after:always;"></div>

In [2]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

#define ARRAY_LEN 10
#define NUM_ARRAYS 5

void *sum_array(void *args) {
    int *nums = (int *)args;
    int *sum = malloc(sizeof(int));
    *sum = 0;
    for (int i = 0; i < ARRAY_LEN; i++) {
        *sum += nums[i];
    }
    return sum;
}

int sum_matrix(int *matrix[]) {
    pthread_t threads[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        pthread_create(&(threads[i]), NULL, &sum_array, matrix[i]);
    }

    int total = 0;
    for (int i = 0; i < NUM_ARRAYS; i++) {
        int *sum;
        pthread_join(threads[i], (void **)(&sum));
        total += *sum;
        free(sum);
    }

    return total;
}

int main() {
    int *matrix[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        matrix[i] = malloc(sizeof(int) * ARRAY_LEN);
        for (int j = 0; j < ARRAY_LEN; j++) {
            matrix[i][j] = i * 100 + j;
        }
    }

    int sum = sum_matrix(matrix);
    printf("%d\n", sum);
}

10225


## Extra practice

* _Assume you are given the following code:_

In [ ]:
#include <ctype.h>
#include <stdio.h>
#include <string.h>
int count(char *str, char ch) {
    int num = 0;
    while (*str != '\0') {
        if (tolower(*str) == ch) {
            num++;
        }
        str++;
    }
    return num;
}
int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Error: provide a string\n");
        return 1;
    }
    char *str = argv[1];
    char *vowels = "aeiou";
    int counts[strlen(vowels)];
    for (int i = 0; i < strlen(vowels); i++) {
        counts[i] = count(str, vowels[i]);
    }

    for (int i = 0; i < strlen(vowels); i++) {
        printf("%c %d\n", vowels[i], counts[i]);
    }
}

* Q4: _Write a function called  `count_wrapper` that has the signature required for a thread function and calls the `count` function. (Hint: you'll need to declare a `struct` that contains all of the parameters required for `count`.)_

* Q5: _Re-write `main` to create/wait for threads that execute `count_wrapper` (instead of calling `count` sequentially)._

In [ ]:
#include <ctype.h>
#include <stdio.h>
#include <string.h>
#include <pthread.h>
int count(char *str, char ch) {
    int num = 0;
    while (*str != '\0') {
        if (tolower(*str) == ch) {
            num++;
        }
        str++;
    }
    return num;
}
struct count_args {
    char *str;
    char ch;
};
void *count_wrapper(void *arg) {
    struct count_args *cargs = (struct count_args *)arg;
    int *result = malloc(sizeof(int));
    *result = count(cargs->str, cargs->ch);
    return result;
}
int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Error: provide a string\n");
        return 1;
    }
    char *str = argv[1];
    char *vowels = "aeiou";

    // Create threads
    struct count_args args[strlen(vowels)];
    pthread_t threads[strlen(vowels)];
    for (int i = 0; i < strlen(vowels); i++) {
        args[i].str = str;
        args[i].ch = vowels[i];
        pthread_create(&(threads[i]), NULL, &count_wrapper, &(args[i]));
    }

    // Wait for threads
    for (int i = 0; i < strlen(vowels); i++) {
        int *ptr;
        pthread_join(threads[i], (void **)(&ptr));
        printf("%c %d\n", vowels[i], *ptr);
    }
}